In [ ]:
####! Large Donut and In-Focus Secondary Offset Sequence Grid: Script for 202110 Observing Run 

In [ ]:
import logging
import sys

BLACK, RED, GREEN, YELLOW, BLUE, MAGENTA, CYAN, WHITE = range(8)

RESET_SEQ = '\033[0m'
COLOR_SEQ = '\033[1;%dm'
BOLD_SEQ = '\033[1m'

COLORS = {
    'WARNING': YELLOW,
    'INFO': WHITE,
    'DEBUG': BLUE,
    'CRITICAL': RED,
    'ERROR': RED
}

class MyLogFormatter(logging.Formatter):
    """ Custom logging formatter that mimics jupyter's server logging. """
    def __init__(self, fmt=" [%(levelname).1s %(asctime)s %(name)s] %(message)s", 
                 datefmt="%Y-%m-%d %H:%M:%S", use_colours=True):

        logging.Formatter.__init__(self, fmt, datefmt=datefmt)
        self.use_colours = use_colours

    @staticmethod
    def color_format(message, levelname, left_char="[", right_char="]"):

        colour = COLOR_SEQ % (30 + COLORS[levelname])

        message = message.replace(left_char, "{:s} {:s}".format(colour, left_char))
        message = message.replace(right_char, "{:s} {:s}".format(right_char, RESET_SEQ))

        return message

    def format(self, record):

        # Call the original formatter class to do the grunt work
        result = logging.Formatter.format(self, record)

        if self.use_colours:
            result = self.color_format(result, record.levelname)

        return result

custom_formatter = MyLogFormatter()
stream_handler = logging.StreamHandler(sys.stdout)
stream_handler.setFormatter(custom_formatter)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.WARNING  # We leave this quiet for now
logger.propagate = False

In [ ]:
import os
import sys
import copy
import asyncio
import logging
import numpy as np

from lsst.ts import salobj
from lsst.ts import idl
from lsst.ts.observatory.control.auxtel import ATCS, LATISS
from lsst.ts.observatory.control.utils.enums import RotType
%config IPCompleter.use_jedi = False

Initiate and configure script

In [ ]:
from lsst.ts.idl.enums.Script import ScriptState
import yaml

In [ ]:
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
script = LatissCWFSAlign(index=333, remotes=True)  # this essentially calls the init method
await script.start_task  # make sure all remotes etc are running

# Find the Target. Elevation 60 around the meridian. 

In [ ]:
#### GOTO 170 since it'll track
target_name = await script.atcs.find_target(180,60,mag_limit=7.0)
print(target_name)

In [ ]:
await script.atcs.slew_object(name = 'HD 138670', rot_type=RotType.PhysicalSky)

In [ ]:
# Take short exposure to check star is centered in the detector
science_image = await script.script.latiss.take_object(exptime=2., n=1, filter='FELH0600',grating='empty_1')
print ('science_image = ' + str(science_image))

In [ ]:
# If not, apply correction pointing offsets to xy to center image in the detector and take another image. 
# await script.atcs.offset_xy(y=140, x=0)
#science_image = await script.latiss.take_object(exptime=2., n=1, filter='FELH0600',grating='empty_1')
#print ('science_image = ' + str(science_image)) 

# CWFS. Execute CWFS on the target in a different notebook. Paste results here 

In [ ]:
# CWFS results 

# LARGE DONUT (Can be done in poorer seeing conditions)

In [ ]:
# Choose target 
await script.atcs.slew_object('HR 807', rot_type=RotType.PhysicalSky)

## Going out of focus (z axis position range = +/- 10 mm) - To get the largest donut possible

In [ ]:
await script.atcs.rem.ataos.cmd_offset.set_start(z=7)

### Take an image and make sure few thousand counts per pixel in the illuminated annulus are present. 

In [ ]:
await script.latiss.take_engtest(exptime=30, n=1, filter='FELH0600',grating='empty_1')

## Secondary offset grid, take 3 20-s images in each position. 
The telescope will offset to compensate for the hexapod offsets so the donut stays in the center of the detector. 

In [ ]:
hexapod_offset_scale = [
    [52.459, 0.0, 0.0],
    [0.0, 50.468, 0.0],
    [0.0, 0.0, 0.0],
]

### For the FELH0600 filter

In [ ]:
for axis in ['x','y','both']:
    for offset in [2.5, 1.0, -1.0, -2.5]:
        if axis == 'x':
            hexapod_offset = [offset,0,0]
        elif axis == 'y':
            hexapod_offset = [0, offset, 0]
        else: #axis == 'both': 
            hexapod_offset = [offset,offset,0]
        
        #print(hexapod_offset)
        tel_offset = np.matmul(hexapod_offset, hexapod_offset_scale) # gives el and negative az!
        #print(tel_offset)
        print('Hexapod offsets', hexapod_offset)
        if axis == 'x':  
            #print(axis, 'axis')
            await script.atcs.rem.ataos.cmd_offset.set_start(x=offset)
        if axis == 'y':  
            #print(axis, 'axis')
            await script.atcs.rem.ataos.cmd_offset.set_start(y=offset)
        if axis == 'both':
            await script.atcs.rem.ataos.cmd_offset.set_start(x=offset)
            await script.atcs.rem.ataos.cmd_offset.set_start(y=offset)
            print(axis, 'axis')

        print('Offsetting the telescope by az offset=', -tel_offset[1], 'and el offset = ', tel_offset[0])
        await script.atcs.offset_azel(az=-tel_offset[1], el=tel_offset[0], relative=True, persistent=True)

        print('Taking 3 30 s engineering images...')
        eng_test = await script.latiss.take_engtest(exptime=20, n=3, filter='FELH0600',grating='empty_1')
        print ('Eng test images with', offset, 'arcsec in', axis, 'axis', str(eng_test))
        
        # Clear telescope and hexapod offsets. 
        print('Clearing telescope and x & y hexapod offsets...')
        await script.atcs.reset_offsets()
        await script.atcs.rem.ataos.cmd_resetOffset.set_start(axis='x')
        await script.atcs.rem.ataos.cmd_resetOffset.set_start(axis='y')

### For the SDSS g

In [ ]:
for axis in ['x','y','both']:
    for offset in [2.5, 1.0, -1.0, -2.5]:
        if axis == 'x':
            hexapod_offset = [offset,0,0]
        elif axis == 'y':
            hexapod_offset = [0, offset, 0]
        else: #axis == 'both': 
            hexapod_offset = [offset,offset,0]
        
        #print(hexapod_offset)
        tel_offset = np.matmul(hexapod_offset, hexapod_offset_scale) # gives el and negative az!
        #print(tel_offset)
        print('Hexapod offsets', hexapod_offset)
        if axis == 'x':  
            #print(axis, 'axis')
            await script.atcs.rem.ataos.cmd_offset.set_start(x=offset)
        if axis == 'y':  
            #print(axis, 'axis')
            await script.atcs.rem.ataos.cmd_offset.set_start(y=offset)
        if axis == 'both':
            await script.atcs.rem.ataos.cmd_offset.set_start(x=offset)
            await script.atcs.rem.ataos.cmd_offset.set_start(y=offset)
            print(axis, 'axis')

        print('Offsetting the telescope by az offset=', -tel_offset[1], 'and el offset = ', tel_offset[0])
        await script.atcs.offset_azel(az=-tel_offset[1], el=tel_offset[0], relative=True, persistent=True)

        print('Taking 3 30 s engineering images...')
        eng_test = await script.latiss.take_engtest(exptime=20, n=3, filter='FELH0600',grating='empty_1')
        print ('Eng test images with', offset, 'arcsec in', axis, 'axis', str(eng_test))
        
        # Clear telescope and hexapod offsets. 
        print('Clearing telescope and x & y hexapod offsets...')
        await script.atcs.reset_offsets()
        await script.atcs.rem.ataos.cmd_resetOffset.set_start(axis='x')
        await script.atcs.rem.ataos.cmd_resetOffset.set_start(axis='y')

# IN- FOCUS SEQUENCE (Requires good seeing) -- Outer edge of NGC 362?

## For the FELH0600 filter

In [ ]:
for axis in ['x','y','both']:
    for offset in [2.5, 1.0, -1.0, -2.5]:
        if axis == 'x':
            hexapod_offset = [offset,0,0]
        elif axis == 'y':
            hexapod_offset = [0, offset, 0]
        else: #axis == 'both': 
            hexapod_offset = [offset,offset,0]
        
        #print(hexapod_offset)
        tel_offset = np.matmul(hexapod_offset, hexapod_offset_scale) # gives el and negative az!
        #print(tel_offset)
        print('Hexapod offsets', hexapod_offset)
        if axis == 'x':  
            #print(axis, 'axis')
            await script.atcs.rem.ataos.cmd_offset.set_start(x=offset)
        if axis == 'y':  
            #print(axis, 'axis')
            await script.atcs.rem.ataos.cmd_offset.set_start(y=offset)
        if axis == 'both':
            await script.atcs.rem.ataos.cmd_offset.set_start(x=offset)
            await script.atcs.rem.ataos.cmd_offset.set_start(y=offset)
            print(axis, 'axis')

        print('Offsetting the telescope by az offset=', -tel_offset[1], 'and el offset = ', tel_offset[0])
        await script.atcs.offset_azel(az=-tel_offset[1], el=tel_offset[0], relative=True, persistent=True)

        print('Taking 3 30 s engineering images...')
        eng_test = await script.latiss.take_engtest(exptime=30, n=3, filter='FELH0600',grating='empty_1')
        print ('Eng test images with', offset, 'arcsec in', axis, 'axis', str(eng_test))
        
        # Clear telescope and hexapod offsets. 
        print('Clearing telescope and x & y hexapod offsets...')
        await script.atcs.reset_offsets()
        await script.atcs.rem.ataos.cmd_resetOffset.set_start(axis='x')
        await script.atcs.rem.ataos.cmd_resetOffset.set_start(axis='y')

### For the SDSS g new installed filter

In [ ]:
for axis in ['x','y','both']:
    for offset in [2.5, 1.0, -1.0, -2.5]:
        if axis == 'x':
            hexapod_offset = [offset,0,0]
        elif axis == 'y':
            hexapod_offset = [0, offset, 0]
        else: #axis == 'both': 
            hexapod_offset = [offset,offset,0]
        
        #print(hexapod_offset)
        tel_offset = np.matmul(hexapod_offset, hexapod_offset_scale) # gives el and negative az!
        #print(tel_offset)
        print('Hexapod offsets', hexapod_offset)
        if axis == 'x':  
            #print(axis, 'axis')
            await script.atcs.rem.ataos.cmd_offset.set_start(x=offset)
        if axis == 'y':  
            #print(axis, 'axis')
            await script.atcs.rem.ataos.cmd_offset.set_start(y=offset)
        if axis == 'both':
            await script.atcs.rem.ataos.cmd_offset.set_start(x=offset)
            await script.atcs.rem.ataos.cmd_offset.set_start(y=offset)
            print(axis, 'axis')

        print('Offsetting the telescope by az offset=', -tel_offset[1], 'and el offset = ', tel_offset[0])
        await script.atcs.offset_azel(az=-tel_offset[1], el=tel_offset[0], relative=True, persistent=True)

        print('Taking 3 30 s engineering images...')
        eng_test = await script.latiss.take_engtest(exptime=30, n=3, filter='FELH0600',grating='empty_1')
        print ('Eng test images with', offset, 'arcsec in', axis, 'axis', str(eng_test))
        
        # Clear telescope and hexapod offsets. 
        print('Clearing telescope and x & y hexapod offsets...')
        await script.atcs.reset_offsets()
        await script.atcs.rem.ataos.cmd_resetOffset.set_start(axis='x')
        await script.atcs.rem.ataos.cmd_resetOffset.set_start(axis='y')

Secondary offset commands:

In [ ]:
#await script.atcs.offset_azel(az=10, el=2, relative=True, persistent=True)
await script.atcs.reset_offsets()


In [ ]:
await script.atcs.rem.ataos.cmd_offset.set_start(z=3)

In [ ]:
await script.atcs.rem.ataos.cmd_offset.set_start(x=1)

In [ ]:
await script.atcs.rem.ataos.cmd_resetOffset.set_start(axis='x')


In [ ]:
await script.atcs.rem.ataos.cmd_setLogLevel.set_start(level=10)

Let's offset all axes

In [ ]:
await script.atcs.rem.ataos.cmd_offset.set_start(y=1)

In [ ]:
await script.atcs.rem.ataos.cmd_offset.set_start(x=1)

In [ ]:
await script.atcs.rem.ataos.cmd_offset.set_start(z=2)

In [ ]:
await script.atcs.point_azel(az=10,el=65)

In [ ]:
await script.atcs.rem.ataos.cmd_resetOffset.set_start(axis='y')
